In [ ]:
# Defining random seeds to enable reproducibility
from numpy.random import seed
seed(1)

import tensorflow as tf
tf.random.set_seed(1)
 
import random
random.seed(1)

import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten
from tensorflow.python.client import device_lib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


def load_dataset():
    """
    Loads the datasets encoded in .pkl files and returns its decoded form.

    Returns
    -------
    list
        A list of n-dimensional arrays representing the subjects samples that will be used to \\
        train the drunkenness classification model.
    ndarray
        A n-dimensional array representing the samples labels.
    """

    print("Loading Sober-Drunk Face Dataset, from Patras University")
    
    # Defining the sample and label sets filenames
    sets = [
        "Insert the x_balanced.pkl file path here",
        "Insert the y_balanced.pkl file path here"   
    ]
    
    # Defining an empty list for storing the decoded dataset
    loaded_datasets = []
 
    # Iterating over the dataset files
    for set_ in sets:
        # Opening the .pkl file in read mode
        with open(set_, 'rb') as file:
            # Appending the decoded dataset to the dataset list
            loaded_datasets.append(pickle.load(file))
    
    # Unpacking the dataset list into individual subsets
    x, y = loaded_datasets
    
    # Converting the label list to the n-dimensional array format
    y_arr= np.array(y)
    
    # Printing the dataset length for sanity check
    print("\nSamples total: {0}".format(len(x)))   
    
    # Slicing the frame sequences of each subject for selecting the
    # frames sampled at each 5 Hz
    x = x[::5]
    y_arr = y_arr[::5]
    
    # Printing the dataset length after slicing for sanity check
    print("\nSamples total after slicing: {0}".format(len(x)))
    
    # Returning the samples set and its respective labels
    return x, y_arr


def min_max_norm(dataset):
    """
    Normalizes the keyframes according to the minimum-maximum norm, \\
    such that pixel values ranges from 0 to 1.

    Parameters
    ----------
    dataset : list
        A list of n-dimensional arrays representing the subjects keyframes.

    Returns
    -------
    ndarray
        A n-dimensional array representing keyframes with pixel values ranging from 0 to 1.
    """

    # Converting the dataset type from list to n-dimensional array
    dataset = np.asarray(dataset, dtype="int16")

    # Finding the keyframes minimum and maximum values
    x_min = dataset.min(axis=(1, 2), keepdims=True)
    x_max = dataset.max(axis=(1, 2), keepdims=True)

    # Applying the minimum-maximum norm to each keyframe
    norm_dataset = (dataset - x_min) / (x_max - x_min)

    # Printing the minimum and maximum values from a given sample for sanity check
    print("\nMinMax normalization")
    print("dataset shape: ", norm_dataset.shape)
    print("min: ", norm_dataset[0].min())
    print("max: ", norm_dataset[0].max())

    # Returning the normalized dataset
    return norm_dataset


def visualize_model_history(hist):
    """
    Displays the training process loss and accuracy history.

    Parameters
    ----------
    hist : dictionary
        A dictionary conataining loss and accuracy values lists from the training and \\
        test sets.
    """

    # Displaying and saving the training and test loss history
    plt.figure()
    plt.plot(hist.history['loss'], linewidth=2, alpha=0.85)
    plt.plot(hist.history['val_loss'], linewidth=2, alpha=0.85)
    sns.despine()
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right', frameon=False)
    plt.savefig("loss-curve.pdf", dpi=600, bbox_inches='tight', pad_inches=0.1)
    
    # Displaying and saving the training and test accuracy history
    plt.figure()
    plt.plot(hist.history['accuracy'], linewidth=2, alpha=0.85)
    plt.plot(hist.history['val_accuracy'], linewidth=2, alpha=0.85)
    sns.despine()
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right', frameon=False)
    plt.savefig("accuracy-curve.pdf", dpi=600, bbox_inches='tight', pad_inches=0.1)
    
    # Showing the training process loss and accuracy history
    plt.show()


def build_model(x_train):
    """
    Builds a keras sequential model and returns it.

    Parameters
    ----------
    x_train : ndarray
        A n-dimensional array representing the training samples.

    Returns
    -------
    Sequential
        A single layer binary classification model.
    """
    
    # Finding the thermal images shape
    imsize = (x_train.shape[1],x_train.shape[2],x_train.shape[3])

    # Defining the pre-trained model new input layer shape
    inp = keras.layers.Input(shape=(imsize[0], imsize[1], imsize[2]))

    # Defining a VGG pre-trained model with new input shape and without dense layers
    base_model = keras.applications.VGG16(include_top=False, weights='imagenet', input_tensor=inp,
                                          input_shape=(imsize[0], imsize[1], imsize[2]))
    
    # Defining the VGG model output as the last pooling layer feature maps
    output = base_model.layers[-1].output
    
    # Changing the model 'trainable' parameter to False,
    # this will prevent the base model from updating its
    # weights during the new dense layers training process
    base_model.trainable = False
    
    print("\n")
    
    # Iterating over the model layers
    for layer in base_model.layers:
        # Freezing the layers weights
        layer.trainable = False
        # Printing the layers 'trainable' parameter for sanity check
        print("{}: {}".format(layer.name, layer.trainable))
    
    # Adding a Flatten layer to the base model output
    flat = Flatten(name='flatten')(output)
    
    # Defining an output layer to classify the received features as sober (0) or drunk (1)
    prediction = keras.layers.Dense(units=1, activation='sigmoid', name='output')
    
    # Stacking the base model convolutional layers and the new dense layer for 
    # drunkenness classification
    x = flat
    x = prediction(x)
    
    # Defining a functional model for transfer learning
    model = models.Model(inp, x)
    
    # Defining the optmization function
    adam = optimizers.Adam(learning_rate=1e-5, amsgrad=False)
    
    # Compiling the model
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=adam,
                  metrics=['accuracy'])
    
    # Printing the model summary
    model.summary()
    
    # Returning the sequential model
    return model


def transfer_learning():
    """
    Performs the transfer learning strategy considering a base model pre-trained \\ 
    with a large scale image dataset.
    """

    # Loading the Sober-Drunk Dataset samples
    x, y = load_dataset()
    
    # Defining the training and test subsets using the random train-test split strategy
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)
      
    # Applying the min-max normalization
    x_train = min_max_norm(x_train)
    x_test = min_max_norm(x_test)

    # Reshaping datsets to the tensor format (channel last)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
    
    # Defining a functional model for transfer learning       
    model = build_model(x_train)
    
    # Fitting the model
    history = model.fit(x=x_train, y=y_train,
                        validation_data=(x_test, y_test),
                        shuffle=False,
                        batch_size=60,
                        epochs=400,   
                        verbose=1)
    
    # Evaluating the model on unseen data
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
    
    # Saving the obtained base drunkenness classification model
    model.save('sober-drunk-cc_vgg16_tl-model_tts_70-30.h5')
    
    print("\nClassification report: ")
    
    # Obtaining the test samples class probabilities
    y_prob = model.predict(x_test)
    # Obtaining the binary label from the model output probabilities
    y_hat = (y_prob > 0.5).astype(int)
    
    # Printing the classification performance report
    report = classification_report(y_test, y_hat, target_names=['sober', 'drunk'])
    print(report)
    
    print("\nConfusion Matrix: ")

    # Printing the confusion matrix
    matrix = confusion_matrix(y_test, y_hat)
    print(matrix)
    
    tn, fp, fn, tp = matrix.ravel()
    print("\nTrue Negatives: ", tn)
    print("False Positives: ", fp)
    print("False Negatives: ", fn)
    print("True Positives: ", tp)
    
    # Displaying the training process history
    visualize_model_history(history)


# Running the transfer learning
transfer_learning()